In [ ]:
# Get Training Data

In [ ]:
from  iesta.machine_learning.dataloader import load_features_df
import iesta.properties

In [ ]:
training_data, feature_dfs = load_features_df()

In [ ]:
liberal_training = feature_dfs['liberal']
liberal_training = liberal_training[liberal_training["effect"]!="okay"]
liberal_training.effect.unique()
liberal_training["binary_effect"].value_counts()
unique_debates = liberal_training.debate_id.unique()

In [ ]:
from sklearn.model_selection import train_test_split


training_debates, testval_debates = train_test_split(unique_debates, test_size=0.3,
                 random_state=42,
                 shuffle=True)

def _add_splits(row, training_debates, testval_debates):
    if row['debate_id'] in training_debates:
        row["classifier_split"] = "train"
    elif row['debate_id'] in testval_debates:
        row["classifier_split"] = "test"
    else:
        assert False
    return row 

liberal_training = liberal_training.apply(_add_splits, args=(training_debates,testval_debates, ),axis=1)

In [ ]:
liberal_training.classifier_split.value_counts()

pd.crosstab(liberal_training.classifier_split, liberal_training.binary_effect)

In [ ]:
import numpy as np
import pandas as pd 
def filter_cols(cols):
    prefices = ["mpqa", "hedge", "emotion", "toxicity", "mpqa", "empath"]
    keep = [c for c in cols if c.split("_")[0] in prefices and not c.endswith("_count")]
    return keep

def filter_feature_df(df: pd.DataFrame, class_col: str = "effect", split_col: str = "classifier_split") : # binary_effect
    feature_cols = filter_cols(df.columns.tolist())
    df_ = df[feature_cols+[class_col, split_col]].copy()
    numeric_cols = df_.select_dtypes(include=np.number).columns.tolist()
    df_ = df_[numeric_cols+[class_col, split_col]]
    
    df_.fillna(0.0, inplace=True)
    #X = df_.values
    #y_str = df[class_].values


    #def to_numeric(x):
    #    return 0 if x == "ineffective" else (1 if x == "effective" else 2)
    #y = [to_numeric(x) for x in y_str]
    
    return df_

In [ ]:


_df = filter_feature_df(liberal_training, class_col = "binary_effect", split_col = "classifier_split")

In [ ]:
_df.binary_effect.value_counts()

In [ ]:
#%pip install -U nlpaf

In [ ]:
from nlpaf.ml.feature_based import FeatureBasedML

trainer = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classification_split", remove_outliers=False, normalize=False)

In [ ]:
dummy = trainer.train_baseline()

In [ ]:
xfold_results = trainer.train_xfold()